In [3]:
import numpy as np 
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy.linalg import det, svd, inv, norm
from scipy import stats
import random

## PCA + KNN Classifier (expression)
def knn_expressions(split_ratio, k_value):
    CLASSES=2
    PCA_DIM=10
    SPLIT_RATIO=split_ratio
    K_VALUE=k_value

    TRAIN=round(200*SPLIT_RATIO)
    TEST=200-TRAIN

    TRAIN_SAMPLES=CLASSES*TRAIN
    TEST_SAMPLES=CLASSES*TEST

    face_data = loadmat('data_mat.mat')
    face_data['face'].size

    ntrl_faces = np.zeros((200,504))
    for i in range(0,600,3):
        ntrl_faces[i//3,:] = face_data['face'][:,:,i].reshape([504])

    smile_faces = np.zeros((200,504))
    for i in range(1,600,3):
        smile_faces[i//3,:] = face_data['face'][:,:,i].reshape([504])


    data = np.zeros((200,504)) 
    data = np.concatenate((ntrl_faces,smile_faces))
    U, sigma, V = svd(data.transpose())
    Y = np.dot(data, U[:,0:PCA_DIM])

    Y_ntrl = Y[0:200]
    Y_smile = Y[200:400]

    random.shuffle(ntrl_faces);
    ntrl_faces_train = Y_ntrl[0:TRAIN]
    ntrl_faces_test = Y_ntrl[TRAIN:]

    random.shuffle(smile_faces);
    smile_faces_train = Y_smile[0:TRAIN]
    smile_faces_test = Y_smile[TRAIN:]

    test_set = np.concatenate((ntrl_faces_test, smile_faces_test), axis=0)
    test_labels = np.concatenate((np.zeros((TEST)), np.ones((TEST))))

    def find_neighbours(dataset, new_data, k):
        distances = []
        for idx, data in enumerate(dataset):
            distances.append(norm(data - new_data))
        return sorted(range(len(distances)), key=lambda i: distances[i])[:k]

    def predict_label(train_labels, neighbours):
        smile=0
        ntrl=0
        for n in neighbours:
            if train_labels[n] == 0:
                ntrl+=1
            if train_labels[n] == 1:
                smile+=1
        if ntrl > smile:
            return 0
        else:
            return 1

    dataset = np.concatenate((ntrl_faces_train, smile_faces_train))
    train_labels = np.concatenate((np.zeros((TRAIN)), np.ones((TRAIN))))

    right=0
    wrong=0
    for idx, test in enumerate(test_set):
        neighbours = find_neighbours(dataset, test, K_VALUE)
        prediction = predict_label(train_labels, neighbours)
        if (prediction == test_labels[idx]):
            right+=1
        else:
            wrong+=1

    print("Split Ratio: {} K-Value: {}".format(SPLIT_RATIO, K_VALUE))
    print("Correct : {} Incorrect : {} Accuracy : {} %".format(right, wrong, right/TEST_SAMPLES * 100))
    return right/TEST_SAMPLES * 100
    
for split_ratio in [0.95, 0.90, 0.80, 0.75]:
    for k_value in [1,2,3,4,5,10,15]:
        knn_expressions(split_ratio,k_value)

Split Ratio: 0.95 K-Value: 1
Correct : 15 Incorrect : 5 Accuracy : 75.0 %
Split Ratio: 0.95 K-Value: 2
Correct : 14 Incorrect : 6 Accuracy : 70.0 %
Split Ratio: 0.95 K-Value: 3
Correct : 14 Incorrect : 6 Accuracy : 70.0 %
Split Ratio: 0.95 K-Value: 4
Correct : 16 Incorrect : 4 Accuracy : 80.0 %
Split Ratio: 0.95 K-Value: 5
Correct : 16 Incorrect : 4 Accuracy : 80.0 %
Split Ratio: 0.95 K-Value: 10
Correct : 16 Incorrect : 4 Accuracy : 80.0 %
Split Ratio: 0.95 K-Value: 15
Correct : 17 Incorrect : 3 Accuracy : 85.0 %
Split Ratio: 0.9 K-Value: 1
Correct : 32 Incorrect : 8 Accuracy : 80.0 %
Split Ratio: 0.9 K-Value: 2
Correct : 30 Incorrect : 10 Accuracy : 75.0 %
Split Ratio: 0.9 K-Value: 3
Correct : 32 Incorrect : 8 Accuracy : 80.0 %
Split Ratio: 0.9 K-Value: 4
Correct : 32 Incorrect : 8 Accuracy : 80.0 %
Split Ratio: 0.9 K-Value: 5
Correct : 34 Incorrect : 6 Accuracy : 85.0 %
Split Ratio: 0.9 K-Value: 10
Correct : 34 Incorrect : 6 Accuracy : 85.0 %
Split Ratio: 0.9 K-Value: 15
Correct : 3